In [1]:
import main

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
import pickle

In [3]:
random_board_pairs = []
with open("2021-07-24-3ply-boards4.pickle", "rb") as f:
    while True:
        random_board_pairs.append(pickle.load(f))

EOFError: Ran out of input

In [4]:
len(random_board_pairs)

28412

In [5]:
game_board_pairs = []
with open("2021-08-04-3ply-boards5.pickle", "rb") as f:
    while True:
        game_board_pairs.append(pickle.load(f))

EOFError: Ran out of input

In [6]:
len(game_board_pairs)

30805

In [68]:
random_inputs = []
random_outputs = []
for p in random_board_pairs:
    random_inputs.append(p[0].get_flipped_3d_array())
    if type(p[1]) != type(3):
        random_outputs.append(p[1])
    else:
        random_outputs.append(np.array(p[1], dtype=np.float32))

In [69]:
game_inputs = []
game_outputs = []
for p in game_board_pairs:
    game_inputs.append(p[0].get_flipped_3d_array())
    if type(p[1]) != type(3):
        game_outputs.append(p[1])
    else:
        game_outputs.append(np.array(p[1], dtype=np.float32))

In [87]:
random_inputs = []
random_outputs = []
for p in random_board_pairs:
    random_inputs.append(p[0].get_flipped_3d_array())
    random_outputs.append(float(p[1]))

In [88]:
game_inputs = []
game_outputs = []
for p in game_board_pairs:
    game_inputs.append(p[0].get_flipped_3d_array())
    game_outputs.append(float(p[1]))

In [89]:
import numpy as np

In [90]:
game_inputs = np.array(game_inputs)

In [91]:
random_inputs = np.array(random_inputs)

In [93]:
game_outputs = np.array(game_outputs)

In [94]:
random_outputs = np.array(random_outputs)

In [95]:
game_outputs.shape

(30805,)

In [12]:
game_outputs[0]

array([0.58236396], dtype=float32)

In [18]:
float(game_outputs[0])

0.5823639631271362

In [19]:
np.array(float(game_outputs[0]))

array(0.58236396)

In [13]:
game_outputs[100]

array([0.41551816], dtype=float32)

In [15]:
game_outputs[100].shape

(1,)

In [16]:
for x in game_outputs:
    if x.shape != (1,):
        print(x)
        break

AttributeError: 'int' object has no attribute 'shape'

In [55]:
for x in game_outputs:
    if x.shape != (1,):
        print(x,x.shape)
        break

1 ()


In [67]:
np.array(1, dtype=np.float32)

array(1., dtype=float32)

In [17]:
x

1

In [56]:
np.array(1)

array(1)

In [22]:
np.ndarray(0.41551816, dtype=np.float32)

TypeError: 'float' object cannot be interpreted as an integer

In [30]:
random_inputs = np.array(random_inputs)

In [29]:
random_outputs = np.array(random_outputs, dtype = object)

In [31]:
random_inputs.shape

(28412, 19, 15, 2)

In [96]:
game_mark = int(len(game_outputs)*0.8)

In [97]:
random_mark = int(len(random_outputs)*0.8)

In [98]:
game_mark

24644

In [99]:
random_mark

22729

In [47]:
train_inputs = random_inputs[0:random_mark] + game_inputs[0:game_mark] + random_inputs[random_mark:] +  game_inputs[game_mark:] 

ValueError: operands could not be broadcast together with shapes (22729,19,15,2) (24644,19,15,2) 

In [100]:
train_inputs = np.concatenate((random_inputs[0:random_mark] , game_inputs[0:game_mark] 
                               , random_inputs[random_mark:] ,  game_inputs[game_mark:])) 

In [101]:
train_outputs = np.concatenate(( random_outputs[0:random_mark],
                                 game_outputs[0:game_mark],
                                random_outputs[random_mark:] ,
                                game_outputs[game_mark:] ))

In [40]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [41]:
model2 = keras.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [42]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
Total params: 19,104
Trainable params: 19,104
Non-trainable params: 0
_________________________________________________________________


In [43]:
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(1, activation='relu'))

In [44]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 10560)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               2703616   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,722,977
Trainable params: 2,722,977
Non-trainable params: 0
_________________________________________________________________


In [45]:
model2.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [103]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0929 - val_loss: 0.0423
Epoch 2/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0381 - val_loss: 0.0365
Epoch 3/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0326 - val_loss: 0.0345
Epoch 4/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0293 - val_loss: 0.0314
Epoch 5/10
1481/1481 [==============================] - 29s 19ms/step - loss: 0.0275 - val_loss: 0.0303
Epoch 6/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0258 - val_loss: 0.0299
Epoch 7/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0250 - val_loss: 0.0301
Epoch 8/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0237 - val_loss: 0.0310
Epoch 9/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0225 - val_loss: 0.0297
Epoch 10/10
1481/1481 [==============================] - 30s 20m

In [104]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1481/1481 [==============================] - 31s 21ms/step - loss: 0.0213 - val_loss: 0.0291
Epoch 2/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0208 - val_loss: 0.0289
Epoch 3/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0199 - val_loss: 0.0285
Epoch 4/10
1481/1481 [==============================] - 29s 19ms/step - loss: 0.0196 - val_loss: 0.0282
Epoch 5/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0189 - val_loss: 0.0282
Epoch 6/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0184 - val_loss: 0.0302
Epoch 7/10
1481/1481 [==============================] - 29s 20ms/step - loss: 0.0179 - val_loss: 0.0288
Epoch 8/10
1481/1481 [==============================] - 29s 19ms/step - loss: 0.0175 - val_loss: 0.0291
Epoch 9/10
1481/1481 [==============================] - 30s 20ms/step - loss: 0.0170 - val_loss: 0.0287
Epoch 10/10
1481/1481 [==============================] - 30s 20m

In [105]:
model2.save('saved_model/2021-08-11-model')

INFO:tensorflow:Assets written to: saved_model/2021-08-11-model/assets


In [51]:
type(3)

int

In [102]:
tf.convert_to_tensor(train_outputs)

<tf.Tensor: shape=(59217,), dtype=float64, numpy=
array([0.75894564, 0.83042955, 0.26181674, ..., 0.80696434, 0.69408554,
       0.82848644])>

In [82]:
train_outputs.shape

(59217,)

In [83]:
train_outputs[0]

array([0.75894564], dtype=float32)

In [84]:
type(train_outputs[0])

numpy.ndarray

In [85]:
np.array([1.1,2.1,3.1])

array([1.1, 2.1, 3.1])

In [86]:
_[0]

1.1